In [1]:
import torch
import torch.nn as nn

import numpy

from model import *
from model.ResNet50 import ResNet50
from model.ResNext50 import ResNext50
from model.MobileNet_V1 import mobilenetv1
from model.VGGNet import VGG
from model.SeResNet50 import seresnet50
from model.ViT import vit
from utils import *

c:\Users\Penguin\anaconda3\envs\prac\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model Structure

In [2]:
import torchsummary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet50().to(device)

torchsummary.summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [3]:
from torchinfo import summary

batch_size = 16
summary(model, input_size=(batch_size, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [16, 10]                  --
├─Sequential: 1-1                        [16, 64, 56, 56]          --
│    └─Conv2d: 2-1                       [16, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                  [16, 64, 112, 112]        128
│    └─ReLU: 2-3                         [16, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                    [16, 64, 56, 56]          --
├─Sequential: 1-2                        [16, 256, 56, 56]         --
│    └─BottleNeck: 2-5                   [16, 256, 56, 56]         --
│    │    └─Sequential: 3-1              [16, 256, 56, 56]         58,112
│    │    └─Sequential: 3-2              [16, 256, 56, 56]         16,896
│    │    └─ReLU: 3-3                    [16, 256, 56, 56]         --
│    └─BottleNeck: 2-6                   [16, 256, 56, 56]         --
│    │    └─Sequential: 3-4              [16, 256, 56, 56]         70,400

## Model Profile

In [7]:
import torch.autograd.profiler as profiler

input = torch.rand(1, 3, 224, 224).cuda()
output = model(input)

# print(output)

with profiler.profile(with_stack=True, profile_memory=True, use_cuda=True) as prof:
    output = model(input)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=6))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Conv Forward        24.99%       3.244ms        99.82%      12.956ms      12.956ms       1.568ms        12.08%      12.959ms      12.959ms          -4 b        -292 b      83.22 Mb     -64.14 Mb             1  
                 aten::batch_norm         3.03%     393.000us        3